# Setup Your Analysis Environment

1) Choose a directory to house your project in: 
```
.../<project-directory>
```


2) Create and activate a fresh Python3 virtual environment (we use Python 3.6.8) there: 
```
$ cd .../<project-directory>
$ python -m virtualenv env 
$ source env/bin/activate
```

3) Download the `cosmicfish` package from Git: 
```
$ git clone git@github.com:ndeporzio/cosmicfish.git
```

4) Install the `cosmicfish` package. Note that its dependencies will install automatically.
```
$ cd cosmicfish
$ pip install . 
```

5) Launch Jupyter and open `tutorial.ipynb` notebook using Jupyter browser
```
$ jupyter notebook
```

6) Create a data folder where the analysis can store spectrum data. This can be anywhere you'd like - you'll specify the path below. 
```
$ mkdir <project-directory>/data
```

7) Install and build CLASS (if you don't already have a build). Note the `cosmicfish` package includes a method for downloading and installing CLASS for you:
```
$ python 
>>> import cosmicfish
>>> cosmicfish.install_class('<project-directory>/class')
>>> exit()
```

# Import & Configure _cosmicfish_

Import the analysis package.

In [ ]:
import cosmicfish as cf 

Import relevant python packages... 

In [ ]:
#Import relevant packages
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

Other setup steps... 

In [ ]:
#Instruct pyplot to use seaborn 
sns.set()

Specify the paths from the setup of your analysis environment above.  

In [ ]:
#Set project, data, CLASS directories 
projectdir = cf.correct_path("~/Desktop/cfworkspace/")
datastore = cf.correct_path("~/Desktop/cfworkspace/data.nosync/")
classpath = os.path.join(projectdir, "class")

Specify the granularity of numerical derivatives.

In [ ]:
dstep = 0.008 #How much to vary parameter to calculate numerical derivative
mu_step = 0.05 #For calculating numerical integral wrt mu between -1 and 1 

# Specify Fiducial Cosmology

In [ ]:
#Setup fiducial cosmology
neutrinofid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.96659,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.11271,
        "tau_reio" : 0.059888,
        "h" : 0.70148,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 3., 
        "m_ncdm" : 0.02, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "relic_fix" : None
        } 


In [ ]:
# For matching the neutrino/relic forecasts, we must modify temp/mass according to equation 44 
# of the Cora/Julian paper. That is: 
# m_chi = m_nu * 3^(1/4) ***where m_nu is the mass of a single neutrino in the degenerate hierarchy
# T_chi = 1.95 * 3^(1/4)

relicfid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.96659,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.11271,
        "tau_reio" : 0.059888,
        "h" : 0.70148,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (1.95 * np.power(3., 1./4.))/2.726, # Units [T_cmb]  Are Julian's matrices at 1.64 K? 
        "m_ncdm" : 0.02 * np.power(3., 1./4.), # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "relic_fix" : "T_ncdm" # Fix T_ncdm or M_ncdm 
        }

relicfid2 = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.96659,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.11271,
        "tau_reio" : 0.059888,
        "h" : 0.70148,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (1.95 * np.power(3., 1./4.))/2.726, # Units [T_cmb]  Are Julian's matrices at 1.64 K? 
        "m_ncdm" : 0.02 * np.power(3., 1./4.), # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "relic_fix" : "m_ncdm" # Fix T_ncdm or M_ncdm 
        }


# Specify Experiment Observational Parameters

In [ ]:
#Setup redshift bins, fiducial relica masses, relic temps to sample
z_table = np.arange(0.65, 1.85, 0.1)
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Demonstrate Convergence 

Before running the forecast, we want to ensure our cosmological parameters are well converged about the points we are interested in using to calculate Fisher matrices. To do so, we can use the `convergence` class of `cosmicfish`. 

We pass to `convergence` some fiducial cosmology, and then it will vary the parameters of that fiducial cosmology by step sizes specified by the user and compute the corresponding power spectrum derivatives. 

All 'varyfactors' are computed relative to the fiducial cosmology. That is:

dtheta = varyfactor * theta_fiducial

**WARNING: This process takes a considerable amount of time (~5 mins per varyfactor w/o prexisting data). Only run when you need to.**

In [ ]:
neutrinoconvergencetest = cf.convergence(
    classpath, # Path to CLASS installation
    datastore, # Path to directory holding CLASS output data
    'neutrino', # 'relic' or 'neutrino' forecasting scheme 
    neutrinofid, # The fiducial cosmology 
    z_table, # Redshift steps in observation
    dNdz, # Redshift noise in observation
    fcoverage_deg=14000, # Sky coverage in observation
    RSD=True, # Use RSD correction to Pm
    FOG=True, # Use FOG correction to Pm
    AP=True, # Use AP correction to PM
    COV=True, #Use AP Change of Variables correction to PM
    varyfactors=[0.001, 0.002, 0.003] # Relative factors used to compute convergence
    )
neutrinoconvergencetest.gen_all_plots() # Display convergence plots

In [ ]:
relicconvergencetest = cf.convergence(
    classpath, # Path to CLASS installation
    datastore, # Path to directory holding CLASS output data
    'relic', # 'relic' or 'neutrino' forecasting scheme 
    relicfid, # The fiducial cosmology 
    z_table, # Redshift steps in observation
    dNdz, # Redshift noise in observation
    fcoverage_deg=14000, # Sky coverage in observation
    RSD=True, # Use RSD correction to Pm
    FOG=True, # Use FOG correction to Pm
    AP=True, # Use AP correction to PM
    COV=True, #Use AP Change of Variables correction to PM
    varyfactors=[0.001, 0.002, 0.003] # Relative factors used to compute convergence
    )
relicconvergencetest.gen_all_plots() # Display convergence plots

# Run Fisher Forecast

In [ ]:
neutrinoforecast = cf.forecast(
    classpath, 
    datastore, 
    'neutrino', 
    neutrinofid, 
    z_table, 
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=0.008,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)

neutrinoforecast.gen_pm()
neutrinoforecast.gen_fisher(mu_step)

In [ ]:
relicforecast = cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    relicfid, 
    z_table, 
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=0.008,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)

relicforecast.gen_pm()
relicforecast.gen_fisher(mu_step)

In [ ]:
relicforecast2 = cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    relicfid2, 
    z_table, 
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=0.008,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)

relicforecast2.gen_pm()
relicforecast2.gen_fisher(mu_step)

# Add CMB Data

In [ ]:
neutrinoforecast.load_cmb_fisher()

In [ ]:
relicforecast.load_cmb_fisher()

# Export Fisher Matrices

In [ ]:
neutrinoforecast.export_matrices()

In [ ]:
relicforecast.export_matrices()

# Light Relic Money Plot

In [ ]:
relicmasses = np.geomspace(0.1, 10.0, 21) * np.power(3., 1./4.)
relicfiducialset = [dict(relicfid, **{'m_ncdm': relicmasses[midx]}) for midx, mval in enumerate(relicmasses)]
relicforecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=0.008,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(relicfiducialset)]

In [ ]:
for fcst in relicforecastset: 
    fcst.gen_pm()
    fcst.gen_fisher(mu_step)

In [ ]:
inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/CMBS4_Fisher_Relic_"
outpath = "/Users/nicholasdeporzio/Desktop/cfworkspace/results/relic/neff_reduced_for_neutrino_forecast_comparison/"

for fidx, fval in enumerate(relicforecastset):
    relicforecastset[fidx].load_cmb_fisher(inpath+str(fidx+1)+".dat")
    relicforecastset[fidx].export_matrices()
    shutil.move("/Users/nicholasdeporzio/Desktop/inv_cmbfisher.mat", outpath+str(fidx+1)+"/inv_cmbfisher.mat")
    shutil.move("/Users/nicholasdeporzio/Desktop/inv_lssfisher.mat", outpath+str(fidx+1)+"/inv_lssfisher.mat")
    shutil.move("/Users/nicholasdeporzio/Desktop/inv_fullfisher.mat", outpath+str(fidx+1)+"/inv_fullfisher.mat")

In [ ]:
relicerromegancdm = [np.sqrt(fval.numpy_full_covariance[6,6]) for fidx, fval in enumerate(relicforecastset)]
relicerromegancdmlss = [np.sqrt(fval.numpy_lss_covariance[6,6]) for fidx, fval in enumerate(relicforecastset)]
relicerromegancdmcmb = [np.sqrt(fval.numpy_cmb_covariance[6,6]) for fidx, fval in enumerate(relicforecastset)]

In [ ]:
plt.figure(figsize=(15,7.5))
plt.semilogx(relicmasses, relicerromegancdm, label="CMB+LSS")
plt.title("Relic Abundance Uncertainty")
plt.xlabel("Relic Mass [eV]")
plt.ylabel(r"$\sigma_\omega$")
plt.legend()
plt.show()

plt.figure(figsize=(15,7.5))
plt.semilogx(relicmasses, relicerromegancdm, label="CMB+LSS")
plt.semilogx(relicmasses, relicerromegancdmcmb, label="CMB")
plt.title("Relic Abundance Uncertainty")
plt.xlabel("Relic Mass [eV]")
plt.ylabel(r"$\sigma_\omega$")
plt.legend()
plt.show()

plt.figure(figsize=(15,7.5))
plt.semilogx(relicmasses, relicerromegancdm, label="CMB+LSS")
plt.semilogx(relicmasses, relicerromegancdmlss, label="LSS")
plt.semilogx(relicmasses, relicerromegancdmcmb, label="CMB")
plt.title("Relic Abundance Uncertainty")
plt.xlabel("Relic Mass [eV]")
plt.ylabel(r"$\sigma_\omega$")
plt.legend()
plt.show()

# Neutrinos Money  Plot

In [ ]:
neutrinomasses = np.geomspace(0.1, 10.0, 21) * (1./3.)
neutrinofiducialset = [dict(neutrinofid, **{'m_ncdm': neutrinomasses[midx]}) 
                       for midx, mval in enumerate(neutrinomasses)]
neutrinoforecastset = [cf.forecast(
    classpath, 
    datastore, 
    'neutrino', 
    fidval, 
    z_table, 
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=0.008,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(neutrinofiducialset)]

In [ ]:
for fcst in neutrinoforecastset: 
    fcst.gen_pm()
    fcst.gen_fisher(mu_step)

In [ ]:
inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/CMBS4_Fisher_Neutrinos.dat"
outpath = "/Users/nicholasdeporzio/Desktop/cfworkspace/results/neutrino/"

for fidx, fval in enumerate(neutrinoforecastset):
    neutrinoforecastset[fidx].load_cmb_fisher(inpath)
    neutrinoforecastset[fidx].export_matrices()
    shutil.move("/Users/nicholasdeporzio/Desktop/inv_cmbfisher.mat", outpath+str(fidx+1)+"/inv_cmbfisher.mat")
    shutil.move("/Users/nicholasdeporzio/Desktop/inv_lssfisher.mat", outpath+str(fidx+1)+"/inv_lssfisher.mat")
    shutil.move("/Users/nicholasdeporzio/Desktop/inv_fullfisher.mat", outpath+str(fidx+1)+"/inv_fullfisher.mat")

In [ ]:
neutrinoerrmncdm = [np.sqrt(fval.numpy_full_covariance[6,6]) for fidx, fval in enumerate(neutrinoforecastset)]
neutrinoerrmncdmlss = [np.sqrt(fval.numpy_lss_covariance[6,6]) for fidx, fval in enumerate(neutrinoforecastset)]
neutrinoerrmncdmcmb = [np.sqrt(fval.numpy_cmb_covariance[6,6]) for fidx, fval in enumerate(neutrinoforecastset)]

neutrinoerromegancdm = 3. * np.array(neutrinoerrmncdm) / 94.
neutrinoerromegancdmlss = 3. * np.array(neutrinoerrmncdmlss) / 94.
neutrinoerromegancdmcmb = 3.* np.array(neutrinoerrmncdmcmb) / 94.

In [ ]:
plt.figure(figsize=(15,7.5))
plt.semilogx(neutrinomasses, neutrinoerromegancdm, label="CMB+LSS")
plt.title("Neutrino Abundance Uncertainty")
plt.xlabel("Sum of Neutrino Masses [eV]")
plt.ylabel(r"$\sigma_\omega$")
plt.legend()
plt.show()

plt.figure(figsize=(15,7.5))
plt.semilogx(neutrinomasses, neutrinoerromegancdm, label="CMB+LSS")
plt.semilogx(neutrinomasses, neutrinoerromegancdmcmb, label="CMB")
plt.title("Neutrino Abundance Uncertainty")
plt.xlabel("Sum of Neutrino Masses [eV]")
plt.ylabel(r"$\sigma_\omega$")
plt.legend()
plt.show()

plt.figure(figsize=(15,7.5))
plt.semilogx(neutrinomasses, neutrinoerromegancdm, label="CMB+LSS")
plt.semilogx(neutrinomasses, neutrinoerromegancdmlss, label="LSS")
plt.semilogx(neutrinomasses, neutrinoerromegancdmcmb, label="CMB")
plt.title("Neutrino Abundance Uncertainty")
plt.xlabel("Sum of Neutrino Masses [eV]")
plt.ylabel(r"$\sigma_\omega$")
plt.legend()
plt.show()

# Load Pre-Existing Results

In [ ]:
inpath = "/Users/nicholasdeporzio/Desktop/cfworkspace/results/relic/neff_reduced_for_neutrino_forecast_comparison/"

relicmasses = np.geomspace(0.1, 10.0, 21) * np.power(3., 1./4.)

relic_lss_fishers = [pd.read_csv(inpath+str(midx+1)+"/inv_lssfisher.mat", sep='\t', header=None, skiprows=1) 
    for midx, mval in enumerate(relicmasses)]
relic_cmb_fishers = [pd.read_csv(inpath+str(midx+1)+"/inv_cmbfisher.mat", sep='\t', header=None, skiprows=1) 
    for midx, mval in enumerate(relicmasses)]
relic_full_fishers = [pd.read_csv(inpath+str(midx+1)+"/inv_fullfisher.mat", sep='\t', header=None, skiprows=1) 
    for midx, mval in enumerate(relicmasses)]

relicerromegancdmlss = [np.sqrt(fval.iloc[6,6]) for fidx, fval in enumerate(relic_lss_fishers)]
relicerromegancdmcmb = [np.sqrt(fval.iloc[6,6]) for fidx, fval in enumerate(relic_cmb_fishers)]
relicerromegancdm = [np.sqrt(fval.iloc[6,6]) for fidx, fval in enumerate(relic_full_fishers)]

In [ ]:
inpath = "/Users/nicholasdeporzio/Desktop/cfworkspace/results/neutrino/"

neutrinomasses = np.geomspace(0.1, 10.0, 21) * (1./3.)

neutrino_lss_fishers = [pd.read_csv(inpath+str(midx+1)+"/inv_lssfisher.mat", sep='\t', header=None, skiprows=1) 
    for midx, mval in enumerate(neutrinomasses)]
neutrino_cmb_fishers = [pd.read_csv(inpath+str(midx+1)+"/inv_cmbfisher.mat", sep='\t', header=None, skiprows=1) 
    for midx, mval in enumerate(neutrinomasses)]
neutrino_full_fishers = [pd.read_csv(inpath+str(midx+1)+"/inv_fullfisher.mat", sep='\t', header=None, skiprows=1) 
    for midx, mval in enumerate(neutrinomasses)]

neutrinoerrmncdmlss = [np.sqrt(fval.iloc[6,6]) for fidx, fval in enumerate(neutrino_lss_fishers)]
neutrinoerrmncdmcmb = [np.sqrt(fval.iloc[6,6]) for fidx, fval in enumerate(neutrino_cmb_fishers)]
neutrinoerrmncdm = [np.sqrt(fval.iloc[6,6]) for fidx, fval in enumerate(neutrino_full_fishers)]

neutrinoerromegancdmlss = 3. * np.array(neutrinoerrmncdmlss) / 94.
neutrinoerromegancdmcmb = 3.* np.array(neutrinoerrmncdmcmb) / 94.
neutrinoerromegancdm = 3. * np.array(neutrinoerrmncdm) / 94.

In [ ]:
plt.figure(figsize=(15,7.5))
plt.semilogx(neutrinomasses, neutrinoerromegancdm, label="CMB+LSS")
plt.title("Neutrino Abundance Uncertainty")
plt.xlabel("Sum of Neutrino Masses [eV]")
plt.ylabel(r"$\sigma_\omega$")
plt.legend()
plt.show()

plt.figure(figsize=(15,7.5))
plt.semilogx(neutrinomasses, neutrinoerromegancdm, label="CMB+LSS")
plt.semilogx(neutrinomasses, neutrinoerromegancdmcmb, label="CMB")
plt.title("Neutrino Abundance Uncertainty")
plt.xlabel("Sum of Neutrino Masses [eV]")
plt.ylabel(r"$\sigma_\omega$")
plt.legend()
plt.show()

plt.figure(figsize=(15,7.5))
plt.semilogx(neutrinomasses, neutrinoerromegancdm, label="CMB+LSS")
plt.semilogx(neutrinomasses, neutrinoerromegancdmlss, label="LSS")
plt.semilogx(neutrinomasses, neutrinoerromegancdmcmb, label="CMB")
plt.title("Neutrino Abundance Uncertainty")
plt.xlabel("Sum of Neutrino Masses [eV]")
plt.ylabel(r"$\sigma_\omega$")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,7.5))
plt.loglog(3*neutrinomasses, erromegancdmlss, label="Relic LSS Error") #Plot relic LSS errors
plt.loglog(3*neutrinomasses, neutrinoerromegancdmlss, label="Neutrino LSS Error")
plt.title("LSS Abundance Uncertainty")
plt.xlabel(r"Equivalent $\Sigma m_\nu$ [eV]")
plt.ylabel(r"$\sigma_\omega$")
plt.grid(True, which="both")
plt.legend()
plt.show()

# Other Diagnostics

### Print power spectra and related values at benchmark k, mu

In [ ]:
neutrinoforecast.print_v_table(k_index=92)
neutrinoforecast.print_P_table(k_index=92, mu_index=20)

In [ ]:
relicforecast.print_v_table(k_index=92)
relicforecast.print_P_table(k_index=92, mu_index=20)

### Compare forecast inferred Pm to CLASS inferred Pm

In [ ]:
#For each z, plot inferred spectrum vs. CLASS data
sns.set()
sns.set_palette("Blues_d", n_colors=2*len(z_table)+1)
plt.figure(figsize=(15, 7.5))
for zidx, zval in enumerate(z_table[0:-1:3]): 
    ps = neutrinoforecast.spectra_mid[zidx]
    plt.semilogx(ps.k_table, neutrinoforecast.Pm[zidx], label=("z = " + str(zval)))
    plt.semilogx(ps.class_pk['k (h/Mpc)']*fid['h'],
             np.array(ps.class_pk['P (Mpc/h)^3'] / np.power(neutrinofid['h'],3.)),
             label='CLASS P(k) Data',
             marker='x',
             linestyle=':')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$P_m$ [Mpc$^3$]')
plt.legend()
plt.show()
#Also plot difference between the two 

In [ ]:
#For each z, plot inferred spectrum vs. CLASS data
sns.set()
sns.set_palette("Blues_d", n_colors=2*len(z_table)+1)
plt.figure(figsize=(15, 7.5))
for zidx, zval in enumerate(z_table[0:-1:3]): 
    ps = relicforecast.spectra_mid[zidx]
    plt.semilogx(ps.k_table, relicforecast.Pm[zidx], label=("z = " + str(zval)))
    plt.semilogx(ps.class_pk['k (h/Mpc)']*relicfid['h'],
             np.array(ps.class_pk['P (Mpc/h)^3'] / np.power(relicfid['h'],3.)),
             label='CLASS P(k) Data',
             marker='x',
             linestyle=':')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$P_m$ [Mpc$^3$]')
plt.legend()
plt.show()
#Also plot difference between the two 

### Compare forecast corrections with MCMC corrections (neutrinoforecast only)

In [ ]:
test_k_table = np.geomspace(0.000001, 0.19663801, 100)
gfacts = np.zeros(len(lightrelic.k_table[4]))
ffacts = np.zeros(len(lightrelic.k_table[4]))
for kidx, kval in enumerate(test_k_table):
    gfacts[kidx] = cf.ggrowth(1.05, kval, 0.70148, 0.02226, 0.11271, 0.06/93.14)
    ffacts[kidx] = cf.fgrowth(0.02226, 0.11271, 0.70148, 1.05)
    
zindex = 4
muindex = 0

testz = lightrelic.z_steps[zindex]
testmu = lightrelic.mu_table[muindex]

z_str_val = '105'
mu_str_val = '-1'

filestr = 'z_' + z_str_val + '_mu_' + mu_str_val

#Print P_g 
print("P_galaxy at: ")
print("z = ",  lightrelic.z_steps[zindex])
print("mu = ", lightrelic.mu_table[muindex])

linda_k_Pm_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_cb_"+filestr+"_relicfast_Nick_test.dat", skiprows=1, usecols=0, delimiter="\t")
linda_Pm_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_cb_"+filestr+"_relicfast_Nick_test.dat",  skiprows=1, usecols=2, delimiter="\t")

linda_k_Pm_norelicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_cb_"+filestr+"_no_relicfast_Nick_test.dat",  skiprows=1, usecols=0, delimiter="\t")
linda_Pm_norelicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_cb_"+filestr+"_no_relicfast_Nick_test.dat",  skiprows=1, usecols=2, delimiter="\t")

linda_k_Pg_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_galaxy_"+filestr+"_relicfast_Nick_test.dat",  skiprows=1, usecols=0, delimiter="\t")
linda_Pg_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_galaxy_"+filestr+"_relicfast_Nick_test.dat",  skiprows=1, usecols=2, delimiter="\t")

linda_k_Pg_norelicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_galaxy_"+filestr+"_no_relicfast_Nick_test.dat",  skiprows=1, usecols=0, delimiter="\t")
linda_Pg_norelicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_galaxy_"+filestr+"_no_relicfast_Nick_test.dat",  skiprows=1, usecols=2, delimiter="\t")

linda_k_rsdfog_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_cb_"+filestr+"_relicfast_Nick_test.dat", skiprows=1, usecols=0, delimiter="\t")
linda_rsd_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_cb_"+filestr+"_relicfast_Nick_test.dat",  skiprows=1, usecols=1, delimiter="\t")
linda_fog_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_cb_"+filestr+"_relicfast_Nick_test.dat",  skiprows=1, usecols=3, delimiter="\t")

linda_k_fg_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/f_and_g_"+filestr+"_relicfast_Nick_test.dat",  skiprows=1, usecols=0, delimiter="\t")
linda_f_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/f_and_g_"+filestr+"_relicfast_Nick_test.dat",  skiprows=1, usecols=1, delimiter="\t")
linda_g_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/f_and_g_"+filestr+"_relicfast_Nick_test.dat",  skiprows=1, usecols=2, delimiter="\t")

#Compare Pm plots
sns.set()
sns.set_palette("Blues_d", n_colors=3)
plt.figure(figsize=(15, 7.5))
plt.loglog(np.array(lightrelic.k_table[zindex]), lightrelic.Pm[zindex], label="Nick/Fisher")
plt.loglog(linda_k_Pm_relicfast_data, linda_Pm_relicfast_data, label="Linda/MCMC w/RelicFast")
plt.loglog(linda_k_Pm_norelicfast_data, linda_Pm_norelicfast_data, label="Linda/MCMC wo/RelicFast")
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$P_m$ [Mpc$^3$]')
plt.title(r'Pm, z = {0:.2f}, mu = {1:.2f}'.format(testz, testmu))
plt.legend()
plt.show()

#Compare Pg plots
sns.set()
sns.set_palette("Blues_d", n_colors=3)
plt.figure(figsize=(15, 7.5))
plt.loglog(np.array(lightrelic.k_table[zindex]), lightrelic.Pg[zindex, :, muindex], label="Nick/Fisher")
plt.loglog(linda_k_Pg_relicfast_data, linda_Pg_relicfast_data, label="Linda/MCMC w/RelicFast")
plt.loglog(linda_k_Pg_norelicfast_data, linda_Pg_norelicfast_data, label="Linda/MCMC wo/RelicFast")
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$P_g$ [Mpc$^3$]')
plt.title(r'Pg, z = {0:.2f}, mu = {1:.2f}'.format(testz, testmu))
plt.legend()
plt.show()

#Also plot difference between the two 
k_vals = np.array(lightrelic.k_table[zindex])
fisher_data_pm = lightrelic.Pm[zindex]
fisher_data_pg = lightrelic.Pg[zindex, :, muindex]
mcmc_data_pm = np.interp(k_vals, linda_k_Pm_relicfast_data, linda_Pm_relicfast_data)
mcmc_data_pg = np.interp(k_vals, linda_k_Pg_relicfast_data, linda_Pg_relicfast_data)

sns.set()
sns.set_palette("Blues_d", n_colors=3)
plt.figure(figsize=(15, 7.5))
plt.semilogx(k_vals, (fisher_data_pm/mcmc_data_pm)-1.0, label="Pm")
plt.semilogx(k_vals, (fisher_data_pg/mcmc_data_pg)-1.0, label="Pg")
plt.title(r'(Fisher/MCMC) - 1, z = {0:.2f}, mu = {1:.2f}'.format(testz, testmu))
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'Ratio')
plt.legend()
plt.show()


#Compare RSD Corrections
sns.set()
sns.set_palette("Blues_d", n_colors=3)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(lightrelic.k_table[zindex]), lightrelic.RSD[zindex, :, muindex], label="Nick/Fisher")
plt.semilogx(linda_k_rsdfog_relicfast_data, linda_rsd_relicfast_data, label="Linda/MCMC w/RelicFast")
plt.title(r'RSD Comparison, z = {0:.2f}, mu = {1:.2f}'.format(testz, testmu))
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'RSD')
plt.legend()
plt.show()

#Compare FOG corrections
sns.set()
sns.set_palette("Blues_d", n_colors=3)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(lightrelic.k_table[zindex]), lightrelic.FOG[zindex, :, muindex], label="Nick/Fisher")
plt.semilogx(linda_k_rsdfog_relicfast_data, linda_fog_relicfast_data, label="Linda/MCMC w/RelicFast")
plt.title(r'FOG Comparison, z = {0:.2f}, mu = {1:.2f}'.format(testz, testmu))
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'FOG')
plt.legend()
plt.show()

#Compare g growth factor
sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(linda_k_fg_relicfast_data, linda_g_relicfast_data, label="MCMC")
plt.semilogx(test_k_table, gfacts, label="Fisher")
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'g')
plt.title(r'g Comparison, z = {0:.2f}, mu = {1:.2f}'.format(testz, testmu))
plt.legend()
plt.show()

#Compare f growth factor
sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(linda_k_fg_relicfast_data, linda_f_relicfast_data, label="MCMC")
plt.semilogx(test_k_table, ffacts, label="Fisher")
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'f')
plt.title(r'f Comparison, z = {0:.2f}, mu = {1:.2f}'.format(testz, testmu))
plt.legend()
plt.show()

# Playground

Confirm that transfer functions look the same for equivalent relic/neutrino cosmologies. . .

In [ ]:
zidx=0

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             neutrinoforecast.spectra_mid[zidx].b_interp_table-relicforecast2.spectra_mid[zidx].b_interp_table, 
             label="Neutrino - Relic")
#plt.semilogx(np.array(relicforecast.spectra_mid[zidx].k_table), 
#             relicforecast.spectra_mid[zidx].b_interp_table, 
#             label="Relic")
plt.title(r'Baryonic Matter Transfer Function, $\Sigma m_\nu = 0.06 eV, z=0.65$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta d_b$')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             neutrinoforecast.spectra_mid[zidx].cdm_interp_table-relicforecast2.spectra_mid[zidx].cdm_interp_table, 
             label="Neutrino - Relic")
#plt.semilogx(np.array(relicforecast.spectra_mid[zidx].k_table), 
#             relicforecast.spectra_mid[zidx].cdm_interp_table, 
#             label="Relic")
plt.title(r'Cold Dark Matter Transfer Function, $\Sigma m_\nu = 0.06 eV, z=0.65$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta d_{cdm}$')
plt.legend()
plt.show()